In [1]:
import arcpy
import os
import openpyxl
import zipfile
import datetime as dt
from copy import deepcopy
from openpyxl import Workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from arcgis.gis import GIS
from arcgis.features import GeoAccessor

##Workbook Variables
excel_path = r"D:\Ground_Truth\EXCEL"
excel_sheet_path = r"D:\Ground_Truth\EXCEL\C3F Training Range Status Report.xlsx"
directory_path = os.path.dirname(os.path.realpath(excel_sheet_path))
wb = openpyxl.load_workbook(excel_sheet_path, data_only=True) # data_only = True prevents formulas from being copied instead of actual cell values -> creates workbook object within Python environment mirroring actual excel workbook
ws = wb.active # sets active sheet to most recent Ground Truth sheet by default/could also specify exact sheet you'd like to update
columnLength = ws.max_row # get total number of data entries

editedReport = 'Edited'+ os.path.basename(excel_sheet_path)
editedReport = editedReport.replace(" ", "")
editedReport = os.path.join(excel_path, editedReport)

columnLength = 150 

##Sets the Organization Value and the Report date 
ORG = ws['F1'].value
report_date = str(ws['F2'].value)

org_lower = ORG.replace(" ", "")
org_lower = org_lower.lower()


## ArcGIS Variables
ArcGIS_path = r"D:\Ground_Truth\ArcGIS"
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx_map = aprx.listMaps("Map2")[0]
arcpy.env.overwriteOutput = True # allows you to rerun program and overwrite previous layer(s)
arcpy.env.preserveGlobalIds = False



## Geodatabases
gt_gdb_path = r"D:\Ground_Truth\ArcGIS\Ground_Truth.gdb"

org_gdb_path = os.path.join(ArcGIS_path, "{}_ground_truth.gdb".format(org_lower))


## Feature Classes
c3f_weekly = os.path.join(gt_gdb_path, 'C3F_GroundTruth')

org_weekly = os.path.join(gt_gdb_path, '{}_Ground_Truth_Weekly'.format(ORG))
org_historical = os.path.join(gt_gdb_path, '{}_Ground_Truth_Historical'.format(ORG))


##Hosted Web Layer - Paths
c3f_endpoint = "https://hostedservices.geo.nga.mil/arcgis/rest/services/Hosted/C3F_GroundTruth/FeatureServer"

org_weekly_endpoint = "https://hostedservices.geo.nga.mil/arcgis/rest/services/Hosted/{}_Ground_Truth_Weekly/FeatureServer".format(ORG)
org_historical_endpoint = "https://hostedservices.geo.nga.mil/arcgis/rest/services/Hosted/{}_Ground_Truth_Historical/FeatureServer".format(ORG)


##Hosted Web Layer - Map
c3f_hosted = "Hosted\\C3F_GroundTruth\\C3F_GroundTruth"

org_hosted_weekly = "Hosted\\{}_Ground_Truth_Weekly\\{}_Ground_Truth_Weekly".format(ORG)
org_hosted_historical = "Hosted\\{}_Ground_Truth_Historical\\{}_Ground_Truth_Historical".format(ORG)




## Sets the 'Year', 'Month', 'Day' & 'Formatted Date'
split_date = report_date.split("/")
month = split_date[0]
if len(month) < 2:
    month = '0'+ month
day = split_date [1]
if len(day) < 2:
    day = '0'+ day
year = split_date[2]

format_date = month + day + year


table = 'Table_' + format_date + '_'+ org_lower # Table_[current sheet] -> name of Table file in ArcGIS
table_path = os.path.join(org_gdb_path, table)

points = 'Point_' + format_date + '_'+ org_lower # Point_[current sheet] -> name of Point Layer file in ArcGIS
points_path = os.path.join(org_gdb_path, points)



##Tupples
# List of Field Names as they are written in ArcGIS, and their corresponding column in the Training Range sheet -> arranged in tuples
fieldNames = [('A3','opArea'), 
              ('B3','field'),
              ('C3', 'msModel'), 
              ('D3','target'), 
              ('E3','snCase'), 
              ('F3','mk5'), 
              ('G3','mag'), 
              ('H3','initDeploy'), 
              ('I3','latitude'), 
              ('J3','longitude'),
              ('K3','locStatus'), 
              ('L3','statusType'),
              ('M3','statusDate'), 
              ('N3','statusDesc'), 
              ('O3','recovDate'),
              ('P3','bottomType'), 
              ('Q3','mineType'),
              ('R3','anchorType'),
              ('S3','depth_ft'), 
              ('T3','tether_ft'),
              ('U3','caseDep_ft'),
              ('V3','caseAlt_ft'),
              ('W3','org'),
              ('X3','reportDate')]


contactList = ['pipe', 
               'Ladder', 
               'MLO', 
               'Box', 
               'Desk', 
               'Pallet', 
               'Distractor', 
               'SLA', 
               'Pipe',
               'Chair', 
               'Z', 
               'Trap', 
               'Drum']# non-mine contacts to delete from excel sheet


dateList = [(1,'Jan'), 
            (2,'Feb'), 
            (3,'Mar'),
            (4, 'Apr'),
            (5,'May'), 
            (6, 'Jun'),
            (7, 'Jul'), 
            (8,'Aug'), 
            (9, 'Sep'),
            (10, 'Oct'),
            (11, 'Nov'),
            (12, 'Dec')]



##Creating all of the Geodatabases

def create_gdb():
    arcpy.env.workspace = ArcGIS_path
    gdbs = arcpy.ListWorkspaces("*", "FileGDB")
    gdb_name = "{}_ground_truth".format(org_lower)
    gdb_check = os.path.join(ArcGIS_path, gdb_name)+ '.gdb'
    if gdb_check in gdbs:
        print(f"!!'{gdb_name}.gdb' Has already been created\n")
    else:
        arcpy.CreateFileGDB_management(ArcGIS_path, gdb_name)
        print(f"**Creating '{gdb_name}.gdb'-\n")
        

        
def rewrite_fields():
    os.chdir(directory_path)
    print('Rewriting field names...')
    for name in fieldNames:
        ws[name[0]].value = name[1] # Field name order is consistent between sheets,
    ## so this should rewrite them to correctly match their ArcGIS counterparts in the Ground Truth layer 
    

    
def format_lat_lon():
    ## Column J4 = start of lat longs
    print('Reformatting Latitude and Longitude values...')
    for cells in ws['I4':('J{}'.format(columnLength))]:
        for cell in cells:
            cellValue = cell.value                       # these worksheets often have an extra decimal point in the lat/long value ->
            for character in cellValue:                  ## this should look through lat longs and automatically replace the extra decimal with a space
                decimal_count = character.count('.')     ### it's always the first decimal that needs to be removed
                if decimal_count > 1:
                    cellValue.replace('.',' ',1)
                    
    for cell in ws['J4':('J{}'.format(columnLength))]:           
        if cell.value.startswith('-') == False:
            cell.value.replace(cell.value,"-{}".format(cell.value))


            
def format_measurements():
    print('Formatting Measurements')
    measurement_columns = ws.iter_cols(min_col=19, max_col=22, min_row = 4, max_row=149)
    for row in measurement_columns: #eplaces strings in columns of int data with zeros -> temp fix until we put in domains? 
        for cell in row:
            if cell.value == 'N/A':
                ws[cell.coordinate] = 0
                
            cellValue_list = list(cell.value)
            if "'" in cellValue_list:
                ws[cell.coordinate] = cell.value.replace("'",'')
                
            if type(cell.value) == float:
                ws[cell.coordinate] = round(cell.value)
                
            if cell.value == None: # ArcPy can't iterate through cells containing "None" value -> replaces with zero
                ws[cell.coordinate] = 0
            
            #print(cell.value)
    print(f"^^Saving {editedReport}\n")
    wb.save(editedReport)
    

    
def arcgis():
    arcpy.env.workspace = org_gdb_path
    
    ##Saving Table to Geodatabase
    print('**Creating ArcGIS Database Table...\n')
    arcpy.ExcelToTable_conversion(Input_Excel_File = editedReport,
                                             Output_Table = table_path,
                                             Sheet = None,
                                             field_names_row = 3,
                                             cell_range = "A3:X150")

##Converting Table to Points
    print('**Creating ArcGIS Point Layer...\n')
    arcpy.defense.CoordinateTableToPoint(in_table= table_path,
                                         out_feature_class = points_path,
                                         x_or_lon_field="Longitude", 
                                         in_coordinate_format = 'DDM_2',
                                         y_or_lat_field="Latitude",                                                   
                                         coordinate_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision")                                                         
    
    
    
##This Function is removing any extra unnamed columns that may tag-along
def delete_columns():
    arcpy.env.workspace = org_gdb_path
    desc = arcpy.Describe(points_path)
    fieldObjList = arcpy.ListFields(points_path, 'COL*')
    fieldDropList = []
    for field in fieldObjList:
        fieldDropList.append(field.name)
    print('--Deleting Extra Columns\n')
    arcpy.DeleteField_management(points, fieldDropList)


                                  
def add_fields():
    print(f"$$Adding Fields to {points}\n")
    arcpy.env.workspace = org_gdb_path
    arcpy.AddFields_management(points_path, 
                          [['depth_m', 'DOUBLE', 'Water Depth (m)'],
                           ['caseDep_m', 'DOUBLE', 'Case Depth (m)'],
                           ['caseAlt_m', 'DOUBLE', 'Case Altitiude (m)'],
                           ['tether_m', 'DOUBLE', 'Tether Length (m)'],
                           ['angle_1', 'TEXT', 'ROV1'],
                           ['angle_2', 'TEXT', 'ROV2'],
                           ['angle_3', 'TEXT', 'ROV3'],
                           ['angle_4', 'TEXT', 'ROV4'],
                           ['sceneview', 'TEXT', '3D Model'],
                           ['image_url', 'TEXT', 'Mineshape Textbook Image'],
                           ['year', 'TEXT', 'Year']])



def calculate_fields():
    arcpy.env.workspace = org_gdb_path
    enforce_domains = True
    code_block = None
    print('Calculating Fields')
    arcpy.CalculateFields_management(points_path, 'PYTHON3',
                                   [['depth_m', "!depth_ft! *.3048"],
                                    ['caseDep_m', "!caseDep_ft! *.3048"],
                                    ['caseAlt_m', "!caseAlt_ft! *.3048"],
                                    ['tether_m', "!tether_ft! *.3048"],
                                    ['year', "!reportDate!.split('/')[-1]"]])

                                     

def remove_rows_weekly():
    arcpy.env.workspace = gt_gdb_path
    print(f"Removing rows from {org_weekly}")
    arcpy.management.DeleteRows(org_weekly)

def remove_rows_c3f():
    arcpy.env.workspace = gt_gdb_path
    print(f"Deleting {ORG} Rows from C3F Ground Truth")
    with arcpy.da.UpdateCursor(c3f_weekly, 'org') as cursor:
        for row in cursor:
            if row[0] == ORG:
                #print(f"Deleting '{ORG}' Rows")
                cursor.deleteRow()
    
def append_weekly():
    arcpy.env.workspace = gt_gdb_path
    print(f"Appending {points} to {org_weekly}")
    arcpy.management.Append(points_path, org_weekly)

def append_historical():
    arcpy.env.workspace = gt_gdb_path
    print(f"Appending {org_weekly} to {org_historical}")
    arcpy.management.Append(org_weekly, org_historical)

def append_c3f():
    arcpy.env.workspace = gt_gdb_path
    print(f"Appending {org_weekly} to C3F Weekly")
    arcpy.management.Append(org_weekly, c3f_weekly)
    
def calculate_images():
    arcpy.env.workspace = gt_gdb_path
    enforce = 'ENFORCE_DOMAINS'
    cb = ""
    print("Calculating Image Fields")
    fc_list = [org_weekly, org_historical, c3f_weekly]
    for fc in fc_list:
        arcpy.CalculateFields_management(fc, 'PYTHON3',
                                        [['angle_1', "!target!+'-1'"],
                                         ['angle_2', '!target!+"-2"'],
                                         ['angle_3', '!target!+"-3"'],
                                         ['angle_4', '!target!+"-4"'],
                                         ['image_url', "!ms_model!"]],
                                          cb,
                                          enforce)

def add_web_layers():
    arcpy.env.workspace = gt_gdb_path
    print(f'Adding "{ORG} GroundTruth Weekly" web layer')
    aprx_map.addDataFromPath(org_weekly_endpoint)
    print(f'Adding "{ORG} GroundTruth Historical" web layer')
    aprx_map.addDataFromPath(org_historical_endpoint)
    print('Adding "C3F GroundTruth"')
    aprx_map.addDataFromPath(c3f_endpoint)
    
def remove_rows_weekly_web():
    arcpy.env.workspace = gt_gdb_path
    print(f"Deleting Rows from {hosted_weekly}(web layer)")
    arcpy.management.DeleteRows(hosted_weekly)[0]

def remove_rows_c3f_web():
    arcpy.env.workspace = gt_gdb_path
    print(f"Deleting {ORG} Rows")
    with arcpy.da.UpdateCursor(hosted_c3f, 'org') as cursor:
        for row in cursor:
            if row[0] == ORG:
                #print(f"Deleting 'MK18' Rows")
                cursor.deleteRow()
     
    
def append_weekly_web():
    arcpy.env.workspace = gt_gdb_path
    print(f"Appending {org_weekly}(gdb) to {org_hosted_weekly}(web layer)")
    arcpy.management.Append(org_weekly, org_hosted_weekly, "TEST")
    
def append_historical_web():
    print(f"Appending {org_weekly}(web layer) to {org_hosted_historical}(web layer)")
    arcpy.management.Append(org_weekly, org_hosted_historical, "TEST")

def append_c3f_web():    
    print(f"Appending {org_weekly}(gdb) to {c3f_hosted}(web layer)")
    arcpy.management.Append(org_weekly, c3f_hosted, "NO_TEST")
    

def save_edits():
    print('Saving Edits')
    edit = arcpy.da.Editor(gt_gdb_path)
    edit.startEditing(True)
    edit.startOperation()
    edit.stopOperation()
    edit.stopEditing(True)

    
    
def remove_web_layers():
    arcpy.env.workspace = gt_gdb_path
    layers = aprx_map.listLayers("Hosted*")
    #print(layers)
    for layer in layers:
        print('Removing Web Layers from map')
        aprx_map.removeLayer(layer)


        
        
if __name__ == '__main__':
    create_gdb()
    ##Sets Default GDB after validating creation
    aprx.defaultGeodatabase = org_gdb_path
    ##Excel
    rewrite_fields()
    format_lat_lon()
    format_measurements()
    ##Organization GDB
    arcgis()
    add_fields()
    calculate_fields()
    ##Sets Default GDB
    aprx.defaultGeodatabase = gt_gdb_path
    ##Ground Truth GDB
    remove_rows_weekly()
    remove_rows_c3f()
    append_weekly()
    append_historical()
    append_c3f()
    calculate_images()
    ##Web
    add_web_layers()
    remove_rows_weekly_web()
    remove_rows_c3f_web()
    append_weekly_web()
    append_historical_web()
    append_c3f_web()
    save_edits()
    remove_web_layers()
    print("Done")
    


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


D:\Ground_Truth\EXCEL\EditedC3FTrainingRangeStatusReport.xlsx
05022022
MMS
5/2/2022
!!'mms_ground_truth.gdb' Has already been created

Rewriting field names...
Reformatting longitude values...
Formatting Measurements
^^Saving D:\Ground_Truth\EXCEL\EditedC3FTrainingRangeStatusReport.xlsx

**Creating ArcGIS Database Table...

**Creating ArcGIS Point Layer...

$$Adding Fields to Point_05022022

MMS
2022
Calculating Fields
Done
